In [3]:
import os
import pandas as pd
import janitor
import numpy as np
pd.set_option('display.max_columns', 100)

DATAPATH = "../../adult-data"        

In [4]:
# (fold cell) Load web browsing data
usecols_web_desktop = ["caseid", "category", "private_domain", "visit_duration", "visit_time_local"]
usecols = ["caseid", "category", "private_domain", "page_duration", "session_start_time"]

df = (pd.concat([
        # Get web_mobile
        pd.read_csv(
            os.path.join(DATAPATH, "output/realityMine_web_mobile_2022-06-01_2022-06-30.csv"),
            usecols=usecols_web_desktop,
            low_memory=False,
        ), 
        # Get web_desktop
        pd.read_csv(
            os.path.join(DATAPATH, "output/realityMine_web_desktop_2022-06-01_2022-06-30.csv"),
            usecols=usecols_web_desktop,
            low_memory=False,
        ),
        # Get web
        (pd.read_csv(
            os.path.join(DATAPATH, "output/realityMine_web_2022-06-01_2022-06-30.csv"),
            usecols=usecols,
            low_memory=False,
        )
         # Renaming columns to be consistent w/ web_mobile & web_desktop
         .rename_column("session_start_time", "visit_time_local")
         .rename_column("page_duration", "visit_duration")
        )
        ])
      .dropna(subset=["caseid", "visit_duration"])
    .assign(caseid=lambda df: df.caseid.astype(object))      
     )
print("df of web browses")
print(f"{len(df)=:,}")
df.head()

df of web browses
len(df)=5,090,441


,caseid,private_domain,category,visit_time_local,visit_duration
0,210362181,liveineverett.com,NaN,2022-05-31 17:19:01,50
1,210362181,wa.gov,"Education, Government",2022-05-31 19:28:41,48
2,210362181,yahoo.com,Economy and Finance,2022-06-01 03:16:33,1
3,210362181,collider.com,Entertainment,2022-06-01 03:29:15,60
4,210362181,yahoo.com,"Education, News and Media",2022-06-01 03:42:43,83


In [5]:
# Zero duration (visit_duration==0)
_nonzeroes = len(df.query("visit_duration!=0"))
print(f"{_nonzeroes=:,}")

_nonzeroes=4,818,516


In [6]:
# Zero duration (visit_duration==0)
_zeroes = len(df.query("visit_duration==0"))
print(f"{_zeroes=:,}")

_zeroes=271,925


In [7]:
# (fold cell) Load the 1200 individual metadata
df_ind_metadata = (
    pd.read_csv(os.path.join(DATAPATH, "SOOD0001_OUTPUT.csv"))
    .assign(
        caseid=lambda df: df.caseid.astype(object),
        birthyr=lambda df: df.birthyr.apply(int),
    )
)
print(f"{len(df_ind_metadata)=}")
df_ind_metadata.head(3)

len(df_ind_metadata)=1200


,caseid,birthyr,gender,race,educ,pid3,pid7,presvote20post,inputstate,region
0,200661421.0,1963,2,1,4,3,3,1,39,2
1,200686597.0,1992,2,6,5,5,8,-1,48,3
2,200953869.0,1959,2,1,5,2,7,2,42,1


In [8]:
ids_webbrowses = df.caseid.unique().tolist()
ids_metadata = df_ind_metadata.caseid.unique().tolist()

len(set(ids_webbrowses) - set(ids_metadata))

361

In [9]:
(pd.DataFrame(set(ids_webbrowses) - set(ids_metadata), columns=["caseid"])
 .to_csv("caseids.csv", index=False)
)